## Imports and configurations

In [1]:
import datetime
from math import sqrt

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.tree import plot_tree

In [2]:
# plot figure size configuration
# plt.rcParams['figure.figsize'] = [25, 5]
sns.set_palette('muted')

In [3]:
data_path = "https://storage.googleapis.com/biosense-ml-data/datasets/heart_disease_uci.csv"

## Read the data

In [4]:
orig_df = pd.read_csv(
    data_path
)

In [5]:
orig_df.isna().sum()

,0
id,0
age,0
sex,0
dataset,0
cp,0
trestbps,59
chol,30
fbs,90
restecg,2
thalch,55


In [6]:
orig_df = orig_df[(orig_df.isna().sum()<100).index]

In [7]:
orig_df.dropna(inplace=True)

In [8]:
orig_df = orig_df.drop(columns=['dataset', 'id'])


### Prepare features and labels - X, y

In [12]:
df = orig_df.copy()

In [10]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,68,Male,asymptomatic,144.0,193.0,True,normal,141.0,False,3.4,flat,2.0,reversable defect,2
300,57,Male,asymptomatic,130.0,131.0,False,normal,115.0,True,1.2,flat,1.0,reversable defect,3
301,57,Female,atypical angina,130.0,236.0,False,lv hypertrophy,174.0,False,0.0,flat,1.0,normal,1
508,47,Male,asymptomatic,150.0,226.0,False,normal,98.0,True,1.5,flat,0.0,reversable defect,1


## Convert categories from string to number

In [13]:
df = pd.get_dummies(df, columns=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'], drop_first=True)

In [14]:
X = df.drop('num', axis=1)
y = df['num']

### Train / test split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Decision Tree - Create and train

In [16]:
tree_model = DecisionTreeClassifier()

In [17]:
tree_model.fit(X_train, y_train)

DecisionTreeClassifier()

## Interpret the tree

In [18]:
plt.rcParams["figure.figsize"] = (120,70)
plot_tree(tree_model, feature_names=X_train.columns, max_depth=2, filled=True, rounded=True)

[Text(0.5, 0.875, 'thal_normal <= 0.5\ngini = 0.66\nsamples = 239\nvalue = [125, 43, 30, 31, 10]'),
 Text(0.25, 0.625, 'oldpeak <= 0.7\ngini = 0.78\nsamples = 106\nvalue = [26, 24, 23, 25, 8]'),
 Text(0.375, 0.75, 'True  '),
 Text(0.125, 0.375, 'chol <= 228.0\ngini = 0.646\nsamples = 32\nvalue = [16, 9, 4, 3, 0]'),
 Text(0.0625, 0.125, '\n  (...)  \n'),
 Text(0.1875, 0.125, '\n  (...)  \n'),
 Text(0.375, 0.375, 'thalch <= 133.5\ngini = 0.775\nsamples = 74\nvalue = [10, 15, 19, 22, 8]'),
 Text(0.3125, 0.125, '\n  (...)  \n'),
 Text(0.4375, 0.125, '\n  (...)  \n'),
 Text(0.75, 0.625, 'age <= 55.5\ngini = 0.42\nsamples = 133\nvalue = [99, 19, 7, 6, 2]'),
 Text(0.625, 0.75, '  False'),
 Text(0.625, 0.375, 'thalch <= 119.5\ngini = 0.211\nsamples = 70\nvalue = [62, 4, 2, 2, 0]'),
 Text(0.5625, 0.125, '\n  (...)  \n'),
 Text(0.6875, 0.125, '\n  (...)  \n'),
 Text(0.875, 0.375, 'thalch <= 111.0\ngini = 0.587\nsamples = 63\nvalue = [37, 15, 5, 4, 2]'),
 Text(0.8125, 0.125, '\n  (...)  \n'),
 Te

## Tree pruning

### Lets evaluate the tree

In [19]:
tree_model.get_depth()

11

In [20]:
tree_model.tree_.n_leaves

np.int64(76)

In [21]:
confusion_matrix(y_test, tree_model.predict(X_test))

array([[31,  4,  0,  0,  0],
       [ 3,  3,  3,  3,  1],
       [ 1,  2,  1,  1,  0],
       [ 0,  3,  0,  1,  0],
       [ 0,  1,  0,  2,  0]])

In [22]:
confusion_matrix(y_train, tree_model.predict(X_train))

array([[125,   0,   0,   0,   0],
       [  0,  43,   0,   0,   0],
       [  0,   0,  30,   0,   0],
       [  0,   0,   0,  31,   0],
       [  0,   0,   0,   0,  10]])

In [23]:
print('Train accuracy:', accuracy_score(y_train, tree_model.predict(X_train)))
print('Test accuracy:', accuracy_score(y_test, tree_model.predict(X_test)))

Train accuracy: 1.0
Test accuracy: 0.6
